#*Importing Libraries*

In [1]:
import numpy as np

#*Function for BP-Sum product decoder*

In [11]:
def bp_sum_product_decoder(H, received_error, max_iter=10, p_error=0.1):

    n = H.shape[1]  # Number of bit nodes
    m = H.shape[0]  # Number of parity checks (check nodes)

    # Compute observed syndrome from received error
    syndrome = H @ received_error % 2
    print("Initially observed Syndrome:", syndrome)

    # Calculate prior LLRs based on received_error and assumed error probability
    prior_LLR = np.log((1 - p_error) / p_error)
    prior_LLRs = np.full(n, prior_LLR)
    prior_LLRs[received_error == 1] = -prior_LLR

    bit_to_check = np.zeros((n, m))
    check_to_bit = np.zeros((m, n))

    # Initialize bit-to-check messages with prior LLRs
    for v in range(n):
        for c in range(m):
            if H[c, v] == 1:
                bit_to_check[v, c] = prior_LLRs[v]

    # BP Iterations
    for iteration in range(max_iter):

        # Check node update: compute check-to-bit messages
        for c in range(m):
            for v in range(n):
                if H[c, v] == 1:
                    product = 1.0
                    for vp in range(n):
                        if vp != v and H[c, vp] == 1:
                            product *= np.tanh(bit_to_check[vp, c] / 2)
                    product = np.clip(product, -0.999999, 0.999999)  # Prevent NaNs
                    check_to_bit[c, v] = 2 * np.arctanh(product)

        # Variable node update: compute bit-to-check messages
        for v in range(n):
            for c in range(m):
                if H[c, v] == 1:
                    sum_msg = prior_LLRs[v]
                    for cp in range(m):
                        if cp != c and H[cp, v] == 1:
                            sum_msg += check_to_bit[cp, v]
                    bit_to_check[v, c] = sum_msg

        # Compute beliefs for each variable node
        beliefs = np.zeros(n)
        for v in range(n):
            total_msg = prior_LLRs[v]
            for c in range(m):
                if H[c, v] == 1:
                    total_msg += check_to_bit[c, v]
            beliefs[v] = total_msg

        # Hard decision based on beliefs
        estimated_error = (beliefs < 0).astype(int)

        # Compute estimated syndrome
        estimated_syndrome = H @ estimated_error % 2

        # Output status
        print(f"\nIteration {iteration + 1}:")
        print("Beliefs:", beliefs)
        print("Estimated Error Pattern:", estimated_error)
        print("Estimated Syndrome:", estimated_syndrome)

        # Check for successful decoding: syndrome must be all zeros
        if np.all(estimated_syndrome == 0):
            print("Decoding successful!")
            return estimated_error, True

    print("Decoding failed after max iterations.")
    return estimated_error, False

#*Example for verification*

In [12]:
#Example 1
H = np.array([[1, 1, 0],[0, 1, 1]])
received_error = np.array([1, 0, 0])
est_error, success = bp_sum_product_decoder(H, received_error, max_iter= 20, p_error=0.1)
print("\nFinal Decoded Error:", est_error)
print("Decoding Success:", success)

Initially observed Syndrome: [1 0]

Iteration 1:
Beliefs: [0.         2.19722458 4.39444915]
Estimated Error Pattern: [0 0 0]
Estimated Syndrome: [0 0]
Decoding successful!

Final Decoded Error: [0 0 0]
Decoding Success: True
